In [40]:
import numpy as np
import pandas as pd
from time import perf_counter
from bisect import insort_left
from collections import deque

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

In [41]:
def random_search(distance_matrix):
    n = len(distance_matrix)
    solution = list(range(n))
    np.random.shuffle(solution)
    
    return solution[:(n//2)]

In [42]:
def calculate_euclidean_distance(node1, node2):
    return np.round(np.sqrt((node1['x'] - node2['x'])**2 + (node1['y'] - node2['y'])**2))

In [43]:
def get_distance_matrix(df, costs=False):
    num_nodes = df.shape[0]
    distance_matrix = np.zeros((num_nodes, num_nodes))
    for i in range(num_nodes):
        node1 = df.iloc[i]
        for j in range(i, num_nodes):
            node2 = df.iloc[j]
            distance = calculate_euclidean_distance(node1, node2)
            if costs:
                distance_matrix[i,j] = distance + node2['cost']
                distance_matrix[j,i] = distance + node1['cost']
            else:
                distance_matrix[i,j] = distance
                distance_matrix[j,i] = distance
    return distance_matrix

In [44]:
def compute_node_swap_delta(solution, swap, distance_matrix, costs):
    new_solution = solution[:]
    n = len(new_solution)

    solution_node, outer_node = swap
    solution_node_idx = new_solution.index(solution_node)
    prev_solution_node, next_solution_node = new_solution[solution_node_idx-1], new_solution[(solution_node_idx+1)%n]

    old_cost = distance_matrix[prev_solution_node][solution_node] + distance_matrix[solution_node][next_solution_node] + \
                costs[solution_node]
    new_cost = distance_matrix[prev_solution_node][outer_node] + distance_matrix[outer_node][next_solution_node] + \
                costs[outer_node]
    
    delta = old_cost - new_cost

    return delta



def compute_new_solution_node_swap(solution, swap):
    new_solution = solution[:]

    solution_node, outer_node = swap
    solution_node_idx = new_solution.index(solution_node)
    
    new_solution[solution_node_idx] = outer_node

    return new_solution

In [45]:
def compute_edge_swap_delta(solution, swap, distance_matrix):
    (edge1_start, edge1_end), (edge2_start, edge2_end) = swap
    idx_edge1_start = solution.index(edge1_start)
    idx_edge2_start = solution.index(edge2_start)
    
    old_distance = distance_matrix[edge1_start][edge1_end] + distance_matrix[edge2_start][edge2_end]
    new_distance = distance_matrix[edge1_start][edge2_start] + distance_matrix[edge1_end][edge2_end]
    
    delta = old_distance - new_distance

    return delta



def compute_new_solution_edge_swap(solution, swap):
    (edge1_start, edge1_end), (edge2_start, edge2_end) = swap
    idx_edge1_start = solution.index(edge1_start)
    idx_edge2_start = solution.index(edge2_start)
    
    if idx_edge1_start < idx_edge2_start:
        new_solution = solution[:idx_edge1_start+1] + solution[idx_edge1_start+1:idx_edge2_start+1][::-1] + \
                       solution[idx_edge2_start+1:] 
    else:
        new_solution = solution[:idx_edge2_start+1] + solution[idx_edge2_start+1:idx_edge1_start+1][::-1] + \
                       solution[idx_edge1_start+1:]
    return new_solution

In [46]:
def get_all_node_moves(solution):
    n = len(solution)
    moves = list()
    outer_nodes = list(set(range(int(n*2))) - set(solution))
    for i in range(len(solution)):
        for j in range(len(outer_nodes)):
            moves.append((solution[i], outer_nodes[j]))   
    return moves


def get_all_edge_moves(solution):
    moves = list()
    for i in range(len(solution) - 1):
        for j in range(i + 2, len(solution) - 1):
            edge1 = (solution[i], solution[i + 1])
            edge2 = (solution[j], solution[j + 1])
            moves.append((edge1, edge2))
    return moves

In [47]:
# def add_move_to_LM(LM, new_move):
#     for idx, move in enumerate(LM):
#         if move[2] < new_move[2]:
#             LM.insert(idx, new_move)
#             return
#     else:
#         LM.append(new_move)


# def get_edge_swaps_with_new_node(solution, node):
#     n = len(solution)
#     idx = solution.index(node)
#     swaps = []

#     edge1 = ( solution[idx-1], solution[idx] )
#     for i in range(idx+1, n):
#         edge2 = (solution[i], solution[(i+1)%n])
#         swap = (edge1, edge2)
#         swaps.append(swap)
#     for i in range(0, idx-2):
#         edge2 = (solution[i], solution[(i+1)%n])
#         swap = (edge1, edge2)
#         swaps.append(swap)
        
#     edge1 = ( solution[idx], solution[(idx+1)%n] )
#     for i in range(idx+2, n):
#         edge2 = (solution[i], solution[(i+1)%n])
#         swap = (edge1, edge2)
#         swaps.append(swap)
#     for i in range(0, idx-1):
#         edge2 = (solution[i], solution[(i+1)%n])
#         swap = (edge1, edge2)
#         swaps.append(swap)
#     return swaps



def add_new_moves_to_LM(solution, solution_set, outer_nodes_set, previous_move, LM, distance_matrix, costs):
    last_move_type, last_move, _ = previous_move
    n = len(solution)
    LM_list = list()
    
    for (move_type, move, delta) in LM:
        
        if last_move_type == 'node_swap' and move_type == 'node_swap':
            if last_move[0] == move[0]:
                new_move = (last_move[1], move[1])
                delta = compute_node_swap_delta(solution, new_move, distance_matrix, costs)
                if delta > 0:
                    LM_list.append(('node_swap', new_move, delta))
            else:
                LM_list.append((move_type, move, delta))
                
        elif last_move_type == 'node_swap' and move_type == 'edge_swap':
            new_move = tuple(tuple(last_move[1] if x == last_move[0] else x for x in inner_tuple) for inner_tuple in move)
            if new_move == move:
                LM_list.append((move_type, move, delta))
            else:
                delta = compute_edge_swap_delta(solution, new_move, distance_matrix)
                if delta > 0:
                    LM_list.append(('edge_swap', new_move, delta))
                    
        elif last_move_type == 'edge_swap' and move_type == 'node_swap':
            if all([i not in move for i in (*last_move[0], *last_move[1])]):
                LM_list.append(('node_swap', move, delta))
            else:
                delta = compute_node_swap_delta(solution, move, distance_matrix, costs)
                if delta > 0:
                    LM_list.append(('node_swap', move, delta))
                    
        elif last_move_type == 'edge_swap' and move_type == 'edge_swap':
            if all([i not in (*move[0], *move[1]) for i in (*last_move[0], *last_move[1])]):
                LM_list.append(('edge_swap', move, delta))
            else:
                delta = compute_edge_swap_delta(solution, move, distance_matrix)
                if delta > 0:
                    LM_list.append(('edge_swap', move, delta))
                    
    LM_list.sort(key=lambda x: x[2], reverse=True)
    return deque(LM_list)

    # if last_move_type == 'node_swap':
    #     removed_node, inserted_node = last_move
    #     # for outer_node in outer_nodes_set:
    #     #     if outer_node != last_move[0]:
    #     #         swap = (inserted_node, outer_node)
    #     #         delta = compute_node_swap_delta(solution, swap, distance_matrix, costs)
    #     #         if delta > 0:
    #     #             add_move_to_LM(LM, ('node_swap', swap, delta))
        
    #     # edge_swaps = get_edge_swaps_with_new_node(solution, inserted_node)
    #     # for swap in edge_swaps:
    #     #     delta = compute_edge_swap_delta(solution, swap, distance_matrix)
    #     #     if delta > 0:
    #     #         add_move_to_LM(LM, ('edge_swap', swap, delta))
        
    #     for idx, (move_type, move, _) in enumerate(LM):
    #         if move_type == 'node_swap':
    #             if move[0] = removed_node:
    #                 new_move = (inserted_node, move[1])
    #                 delta = compute_node_swap_delta(solution, new_move, distance_matrix, costs)
    #                 if delta > 0:
    #                     add_move_to_LM(LM, ('node_swap', delta, delta))

In [48]:
def move_is_applicable(solution, solution_set, outer_nodes_set, move, distance_matrix, costs):
    move_type, move, _ = move
    
    if move_type == 'node_swap':
        node_to_remove, node_to_insert = move
        if node_to_remove not in solution_set:
            return False
        if node_to_insert not in outer_nodes_set:
            return False
        if compute_node_swap_delta(solution, move, distance_matrix, costs) <= 0:
            return False
        
    elif move_type == 'edge_swap':
        (node1, node2), (node3, node4) = move
        if node1 not in solution_set or node2 not in solution_set or node3 not in solution_set or node4 not in solution_set:
            return False
        if len(set([node1, node2, node3, node4])) != 4:
            return False
        if compute_edge_swap_delta(solution, move, distance_matrix) <= 0:
            return False
        
    return True

In [49]:
def steepest_local_search_deltas(solution, distance_matrix, costs):
    LM = []
    first_iteration_flag = True
    solution_set = set(solution)
    outer_nodes_set = set(range(len(solution)*2)) - solution_set
    
    while True:

        if first_iteration_flag:
            
            all_node_moves = get_all_node_moves(solution)
            for node_move in all_node_moves:
                delta = compute_node_swap_delta(solution, node_move, distance_matrix, costs)
                if delta > 0:
                    LM.append( ('node_swap', node_move, delta) )
                    
            all_edge_moves = get_all_edge_moves(solution)
            for edge_move in all_edge_moves:
                delta = compute_edge_swap_delta(solution, edge_move, distance_matrix)
                if delta > 0:
                    LM.append( ('edge_swap', edge_move, delta) )
                    
            LM.sort(key=lambda x: x[2], reverse=True)
            LM = deque(LM)
            move = LM.popleft()
            previous_move = move
            if move[0] == 'node_swap':
                solution = compute_new_solution_node_swap(solution, move[1])
                solution_set.remove(move[1][0])
                solution_set.add(move[1][1])
                outer_nodes_set.remove(move[1][1])
                outer_nodes_set.add(move[1][0])
            elif move[0] == 'edge_swap':
                solution = compute_new_solution_edge_swap(solution, move[1])
            first_iteration_flag = False
        
        else:
            LM = add_new_moves_to_LM(solution, solution_set, outer_nodes_set, previous_move, LM, distance_matrix, costs)
            while True:
                if len(LM) == 0:
                    return solution
                move = LM.popleft()
                if move_is_applicable(solution, solution_set, outer_nodes_set, move, distance_matrix, costs):
                    previous_move = move
                    if move[0] == 'node_swap':
                        solution = compute_new_solution_node_swap(solution, move[1])
                        solution_set.remove(move[1][0])
                        solution_set.add(move[1][1])
                        outer_nodes_set.remove(move[1][1])
                        outer_nodes_set.add(move[1][0])
                    elif move[0] == 'edge_swap':
                        solution = compute_new_solution_edge_swap(solution, move[1])
                    break

In [50]:
df = pd.read_csv('../data/TSPA.csv', names=['x', 'y', 'cost'], sep=';')
costs = df.cost
distance_matrix = get_distance_matrix(df)
distance_matrix_with_costs = get_distance_matrix(df, True)
solutionn = random_search(distance_matrix)

In [51]:
total_cost = sum(distance_matrix_with_costs[solutionn[i]][solutionn[i+1]] for i in range(len(solutionn) - 1))
total_cost += distance_matrix_with_costs[solutionn[-1]][solutionn[0]]
total_cost

278432.0

In [52]:
solutionnn = steepest_local_search_deltas(solutionn, distance_matrix, costs)

In [53]:
total_cost = sum(distance_matrix_with_costs[solutionnn[i]][solutionnn[i+1]] for i in range(len(solutionnn) - 1))
total_cost += distance_matrix_with_costs[solutionnn[-1]][solutionnn[0]]
total_cost

210572.0

In [62]:
from sortedcontainers import SortedSet

x = SortedSet([1,4,5,7,2,1,6,7,4,3,2])


SortedSet([1, 2, 3, 4, 5, 6, 7])

In [69]:
x = SortedSet([ (3, ((1,2), (3,4)), 'edge'), (2, (5,6), 'node'),  (1, (7,8), 'node'), (3, ((-11,2), (3,4)), 'edge') ])
x

SortedSet([(1, (7, 8), 'node'), (2, (5, 6), 'node'), (3, ((-11, 2), (3, 4)), 'edge'), (3, ((1, 2), (3, 4)), 'edge')])

In [54]:
# columns = ["Algorithm", "TSPA", "TSPB", "TSPC", "TSPD"]
# cost_df = pd.DataFrame(columns=columns)
# time_df = pd.DataFrame(columns=columns)
# best_solutions = {}

# instances = ['TSPA', 'TSPB', 'TSPC', 'TSPD']
# path = "../data/"

# algo_name = 'Steepest-CandidateMovesEdges-Random'
# new_row = pd.DataFrame({columns[0]: algo_name}, index=[0])
# cost_df = pd.concat([cost_df, new_row], ignore_index=True)
# time_df = pd.concat([time_df, new_row], ignore_index=True)
# best_solutions_tmp = {}

# for instance in instances:
#     file_name = f'{path}{instance}.csv'
#     df = pd.read_csv(file_name, names=['x', 'y', 'cost'], sep=';')
#     distance_matrix = get_distance_matrix(df)
#     distance_matrix_with_costs = get_distance_matrix(df, True)
#     costs = df.cost.values
#     total_costs, solutions, times = list(), list(), list()
#     candidate_moves = get_candidate_moves(distance_matrix, costs)
#     print(algo_name, instance)
#     for i in range(200):
#         print(i)
#         solution = random_search(distance_matrix_with_costs)
#         start_time = perf_counter()
#         solution = steepest_local_search(solution, distance_matrix, costs, candidate_moves)
#         total_cost = sum(distance_matrix_with_costs[solution[i]][solution[i+1]] for i in range(len(solution) - 1))
#         total_cost += distance_matrix_with_costs[solution[-1]][solution[0]]
#         end_time = perf_counter()
#         total_costs.append(total_cost)
#         solutions.append(solution)
#         times.append(round(end_time - start_time, 3))
#     best_solution_idx = np.argmin(total_costs)
#     best_solutions_tmp[file_name] = solutions[best_solution_idx]
#     cost_df.at[cost_df.index[-1], instance] = f'{np.mean(total_costs)} ({np.min(total_costs)} - {np.max(total_costs)})'
#     time_df.at[time_df.index[-1], instance] = f'{round(np.mean(times), 3)} ({np.min(times)} - {np.max(times)})'
#     print(f'{np.mean(total_costs)} ({np.min(total_costs)} - {np.max(total_costs)})')
#     print(f'{np.mean(times)} ({np.min(times)} - {np.max(times)})')
#     print(solutions[best_solution_idx])
#     display(cost_df)
#     display(time_df)
# best_solutions[algo_name] = best_solutions_tmp

In [55]:
# cost_df.to_csv('costs.csv')

In [56]:
# time_df.to_csv('times.csv')

In [57]:
# display(cost_df)

In [58]:
# display(time_df)

In [59]:
# best_solutions = {}
# best_solutions['algo'] = {}
# best_solutions['algo']['../data/TSPA.csv'] = solution

In [60]:
# for algo in best_solutions:
#     fig, ax = plt.subplots(2, 2, figsize=(25, 25))
#     ax = ax.flatten()
#     idx = 0
#     fig.suptitle(algo, fontsize=40)
#     for instance in best_solutions[algo]:
#         solution = best_solutions[algo][instance]
#         df = pd.read_csv(instance, names=['x', 'y', 'cost'], sep=';')
#         weights = df['cost']
#         cmap = plt.cm.viridis
#         norm = mcolors.Normalize(vmin=min(weights), vmax=max(weights))
#         ax[idx].set_title(instance.replace('../data/', '').replace('.csv', ''), fontsize=30)
#         for i in range(len(df)):
#             x, y, cost = df.iloc[i]['x'], df.iloc[i]['y'], df.iloc[i]['cost']
#             ax[idx].plot(x, y, "o", markersize=20, color=cmap(norm(cost)))
#         for i in range(len(solution)-1):
#             x, y, cost = df.iloc[solution[i]]['x'], df.iloc[solution[i]]['y'], df.iloc[solution[i]]['cost']
#             x_next, y_next = df.iloc[solution[i+1]]['x'], df.iloc[solution[i+1]]['y']
#             ax[idx].plot((x, x_next), (y, y_next), "-", color='black')
#         x, y = df.iloc[solution[0]]['x'], df.iloc[solution[0]]['y']
#         ax[idx].plot((x, x_next), (y, y_next), "-", color='black')
        
#         axins = ax[idx].inset_axes([1.05, 0.1, 0.05, 0.6], transform=ax[idx].transAxes)
#         gradient = np.linspace(0, 1, 256).reshape(-1, 1)

#         axins.imshow(gradient, aspect='auto', cmap=cmap, origin='lower', extent=[0, 1, min(weights), max(weights)])
#         axins.xaxis.set_visible(False)
        
#         idx += 1
        
#     # plt.savefig(f'./plots/{algo}.png', dpi=300)
#     plt.show()